<a href="https://colab.research.google.com/github/UrviSinghal28/ML-CaPsule/blob/master/Gender%20Pay%20Gap%20Analysis/analyze_gender_pay_gap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Analysing Gender Pay Gap**

# *Importing Libraries*

In [ ]:
pip install -U pandas-profiling

In [ ]:
!pip install pyyaml==5.4.1

In [ ]:
import numpy as np # linear algebra
import pandas as pd
import plotly.graph_objs as go

%matplotlib inline
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.offline as py
import seaborn as sns

import pandas_profiling

import random
plt.style.use("fivethirtyeight")

import warnings
warnings.filterwarnings('ignore')

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# *Loading Dataset*

In [4]:
df = pd.read_csv('/content/Glassdoor Gender Pay Gap.csv')

# *Basic Data Analysis*

In [5]:
df.profile_report()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
df.head()

,JobTitle,Gender,Age,PerfEval,Education,Dept,Seniority,BasePay,Bonus
0,Graphic Designer,Female,18,5,College,Operations,2,42363,9938
1,Software Engineer,Male,21,5,College,Management,5,108476,11128
2,Warehouse Associate,Female,19,4,PhD,Administration,5,90208,9268
3,Software Engineer,Male,20,5,Masters,Sales,4,108080,10154
4,Graphic Designer,Male,26,5,Masters,Engineering,5,99464,9319


In [7]:
df.describe()

,Age,PerfEval,Seniority,BasePay,Bonus
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,41.393000,3.037000,2.971000,94472.653000,6467.161000
std,14.294856,1.423959,1.395029,25337.493272,2004.377365
min,18.000000,1.000000,1.000000,34208.000000,1703.000000
25%,29.000000,2.000000,2.000000,76850.250000,4849.500000
50%,41.000000,3.000000,3.000000,93327.500000,6507.000000
75%,54.250000,4.000000,4.000000,111558.000000,8026.000000
max,65.000000,5.000000,5.000000,179726.000000,11293.000000


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   JobTitle   1000 non-null   object
 1   Gender     1000 non-null   object
 2   Age        1000 non-null   int64 
 3   PerfEval   1000 non-null   int64 
 4   Education  1000 non-null   object
 5   Dept       1000 non-null   object
 6   Seniority  1000 non-null   int64 
 7   BasePay    1000 non-null   int64 
 8   Bonus      1000 non-null   int64 
dtypes: int64(5), object(4)
memory usage: 70.4+ KB


**The dataset contains 1000 rows and 9 columns. It has 4 categorical columns and 5 numerical.**

In [9]:
df['TotalPay'] = df['BasePay'] + df['Bonus']

In [10]:
print('Job Titles: ')
df.JobTitle.value_counts()

Job Titles: 


Marketing Associate    118
Software Engineer      109
Financial Analyst      107
Data Scientist         107
Graphic Designer        98
IT                      96
Sales Associate         94
Driver                  91
Warehouse Associate     90
Manager                 90
Name: JobTitle, dtype: int64

**From this we can see that maximum number of people have the job of marketing associate (118) and minimum the job of manager and warehouse associate (90 each).**

In [11]:
print('Education Level: ')
df.Education.value_counts()

Education Level: 


High School    265
Masters        256
College        241
PhD            238
Name: Education, dtype: int64

**Most of the people are high school graduates (265) and 238 of them have done their PhD.**

# *Visualisation*

In [12]:
gender = df.groupby('Gender').count()

fig = go.Figure(data=[go.Bar(
            x = gender.index,
            y = gender['JobTitle'],
            #text=y,
            width=0.3,
            textposition='auto',
            marker=dict()
 )])

fig.data[0].marker.line.width = 1
fig.data[0].marker.line.color = "black"
fig.update_layout(yaxis=dict(title=''),width=500,height=500,
                  title= 'No of Male and Female Job Entries on the Dataset',
                  xaxis=dict(title='Gender'))
fig.show()

**From this we can see that number of female job entries are 468 and number of male ones are 532. Therefore males have more jobs.**

In [13]:
gender_dept = df.groupby(['Dept','Gender']).size().reset_index(name='counts')
fig = px.bar(gender_dept, x='Dept',y='counts',color='Gender',barmode='group', title ='Count Gender per Department')
fig.show()

**From this we can see that administration and sales department have almost equal gender radio while operations has the largest difference between males and females.**

In [14]:
title = pd.get_dummies(df, columns=['Gender']).groupby('Dept').sum()

female = go.Pie(labels=title.index,values=title['Gender_Female'],name="Female",hole=0.5,domain={'x': [0,0.46]})
male = go.Pie(labels=title.index,values=title['Gender_Male'],name="Male",hole=0.5,domain={'x': [0.52,1]})

layout = dict(title = 'Department Distribution', font=dict(size=14), legend=dict(orientation="h"),
              annotations = [dict(x=0.2, y=0.5, text='Female', showarrow=False, font=dict(size=20)),
                             dict(x=0.8, y=0.5, text='Male', showarrow=False, font=dict(size=20)) ])

fig = dict(data=[female, male], layout=layout)
py.iplot(fig)

**This shows that most of the females are engaged in sales work (21.6%) and least in management (18.6%). For males, highest is operations (21.4%) and least is administration (18.4%).**

In [15]:
gender_job = df.groupby(['JobTitle','Gender']).size().reset_index(name='counts')
fig = px.bar(gender_job, x='JobTitle',y='counts',color='Gender',barmode='group', title ='Count Gender per JobTitle')
fig.show()

**In software engineering jobs, men have a much higher population and for marketing associate, it is the females who dominate.**

In [16]:
title = pd.get_dummies(df, columns=['Gender']).groupby('JobTitle').sum()

female = go.Pie(labels=title.index,values=title['Gender_Female'],name="Female",hole=0.5,domain={'x': [0,0.46]})
male = go.Pie(labels=title.index,values=title['Gender_Male'],name="Male",hole=0.5,domain={'x': [0.52,1]})

layout = dict(title = 'Job Title Distribution', font=dict(size=14), legend=dict(orientation="h"),
              annotations = [dict(x=0.2, y=0.5, text='Female', showarrow=False, font=dict(size=20)),
                             dict(x=0.8, y=0.5, text='Male', showarrow=False, font=dict(size=20)) ])

fig = dict(data=[female, male], layout=layout)
py.iplot(fig)

**This shows that most of the females are employed as a marketing associate (22.9%) and least as software engineer (1.71%). For males, highest is software engineer (19%) and least is marketing associate (2.07%).**